In [25]:
import numpy as np
import qiskit as qk
from qiskit.quantum_info import SparsePauliOp, Pauli

H_mat = np.array([[ -3,  -4,   2,  -3],
 [ -4,  -2, -10,   1],
 [  2, -10,   2,  -1],
 [ -3,   1,  -1,  -9]])

H = SparsePauliOp.from_operator(H_mat)
groups = H.group_commuting()
groups

[SparsePauliOp(['IX', 'XI', 'XX', 'II'],
               coeffs=[-2.5+0.j,  1.5+0.j, -6.5+0.j, -3. +0.j]),
 SparsePauliOp(['IZ', 'XZ'],
               coeffs=[2.5+0.j, 0.5+0.j]),
 SparsePauliOp(['YY', 'ZX'],
               coeffs=[-3.5+0.j, -1.5+0.j]),
 SparsePauliOp(['ZI', 'ZZ'],
               coeffs=[ 0.5+0.j, -3. +0.j])]

In [26]:
for group in groups:
    print(group.paulis)
    print(sum([1 for s in group.paulis.to_labels() if 'I' in s]))


['IX', 'XI', 'XX', 'II']
3
['IZ', 'XZ']
1
['YY', 'ZX']
0
['ZI', 'ZZ']
1


In [61]:
def ansatz(params, target = None):
    """Create the ansatz with a parameterized quantum circuit."""
    state = qk.QuantumCircuit(2,2)
    state.ry(params[0], 0)
    state.ry(params[0], 1)
    state.cx(0, 1)
    state.ry(params[1], 0)
    state.ry(params[1], 1)

    if target is not None:
        state = target

    return state

In [ ]:
from qiskit_ibm_runtime import Estimator
from qiskit_aer import AerSimulator

def evaluate_energy(params, pauli_list, shots=1_000):
    """Evaluate the expectation value of a Pauli operator given a quantum circuit."""
    backend = AerSimulator(method='statevector')
    estimator = Estimator(backend, options={"default_shots": shots})

    quantum_circuit = ansatz(params)
    
    
    job = estimator.run([(quantum_circuit, pauli_list)])
    measured_energy = job.result()[0].data.evs

    return measured_energy.item()

In [63]:
evaluate_energy([0.1, 0.2], H, shots=1000)

-3.7272727272727266

In [64]:
from scipy.optimize import minimize

results = minimize(evaluate_energy, [0, 0], args=(H, 500), method='COBYLA')

opt_angles = results.x
opt_energy = results.fun

print('Solution angles:', opt_angles)
print('Optimal energy:', opt_energy)

Solution angles: [1.16670486 1.18643793]
Optimal energy: -10.143999999999998
